In [1]:
import pandas as pd

# For Project 2 Decision Trees

Information Gain Tree

In [3]:
import numpy as np
import pandas as pd
eps = np.finfo(float).eps
from numpy import log2 as log

In [4]:
dataset = {'Outlook':['Sunny','Sunny','Overcast','Rain','Rain','Rain','Overcast','Sunny','Sunny','Rain', 'Sunny', 'Overcast', 'Overcast', 'Rain'],
       'Temperature':['Hot','Hot','Hot','Mild','Cool','Cool','Cool','Mild','Cool','Mild', 'Mild', 'Mild', 'Hot', 'Mild'],
       'Humidity':['High','High','High','High','Normal','Normal','Normal','High','Normal','Normal', 'Normal', 'High', 'Normal', 'High'],
       'Wind':['Weak','Strong','Weak','Weak','Weak','Strong','Strong','Weak','Weak','Weak', 'Strong', 'Strong', 'Weak', 'Strong'],
        'PlayTennis':['No','No','Yes','Yes','Yes','No','Yes','No','Yes','Yes', 'Yes', 'Yes', 'Yes', 'No']}
df = pd.DataFrame(dataset,columns=['Outlook','Temperature','Humidity','Wind', 'PlayTennis'])
print(df)

     Outlook Temperature Humidity    Wind PlayTennis
0      Sunny         Hot     High    Weak         No
1      Sunny         Hot     High  Strong         No
2   Overcast         Hot     High    Weak        Yes
3       Rain        Mild     High    Weak        Yes
4       Rain        Cool   Normal    Weak        Yes
5       Rain        Cool   Normal  Strong         No
6   Overcast        Cool   Normal  Strong        Yes
7      Sunny        Mild     High    Weak         No
8      Sunny        Cool   Normal    Weak        Yes
9       Rain        Mild   Normal    Weak        Yes
10     Sunny        Mild   Normal  Strong        Yes
11  Overcast        Mild     High  Strong        Yes
12  Overcast         Hot   Normal    Weak        Yes
13      Rain        Mild     High  Strong         No


In [34]:
# calculate entropy for final desicion
entropy_node = 0  #Initialize Entropy

values = df.PlayTennis.unique()  
print('values', values)
for value in values:
    #fraction = df.Eat.value_counts()[value]/len(df.Eat)
    fraction = df.PlayTennis.value_counts()[value]/len(df.PlayTennis)
    print(' probability', fraction)
    entropy_node += -fraction*np.log2(fraction)
    print('entropy of node', entropy_node)

values ['No' 'Yes']
 probability 0.35714285714285715
entropy of node 0.5305095811322292
 probability 0.6428571428571429
entropy of node 0.9402859586706311


In [16]:
# entropy calculation for specific attribute
attribute = 'Wind'

#target_variables = df.Eat.unique()
target_variables = df.PlayTennis.unique()  #This gives all 'Yes' and 'No'
print('target variables',target_variables)
variables = df[attribute].unique()    #This gives different features in that attribute (like 'Sweet')
print('variables', variables)
entropy_attribute = 0
for variable in variables:
    entropy_each_feature = 0
    for target_variable in target_variables:
        
        #num = len(df[attribute][df[attribute]==variable][df.Eat ==target_variable]) #numerator
        num = len(df[attribute][df[attribute]==variable][df.PlayTennis ==target_variable]) #numerator
        print('num', num)
        den = len(df[attribute][df[attribute]==variable])  #denominator
        print('den', den)
        fraction = num/(den+eps)  #pi
        print('fraction', fraction)
        entropy_each_feature += -fraction*log(fraction+eps) #This calculates entropy for one feature like 'Sweet'
        print('feature entropy', entropy_each_feature)
    fraction2 = den/len(df)
    entropy_attribute += fraction2*entropy_each_feature   #Sums up all the entropy ETaste
    print('entropy of attribute', variable, entropy_attribute)

target variables ['No' 'Yes']
variables ['Weak' 'Strong']
num 2
den 8
fraction 0.25
feature entropy 0.49999999999999967
num 6
den 8
fraction 0.75
feature entropy 0.8112781244591322
entropy of attribute Weak 0.46358749969093266
num 3
den 6
fraction 0.5
feature entropy 0.49999999999999967
num 3
den 6
fraction 0.5
feature entropy 0.9999999999999993
entropy of attribute Strong 0.892158928262361


In [21]:
def find_entropy(df):
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name
    entropy = 0
    values = df[Class].unique()
    for value in values:
        print('value in find entropy function', value)
        fraction = df[Class].value_counts()[value]/len(df[Class])
        entropy += -fraction*np.log2(fraction)
    return entropy
  
  
def find_entropy_attribute(df,attribute):
  print('attribute', attribute)
  Class = df.keys()[-1]   #To make the code generic, changing target variable class name
  target_variables = df[Class].unique()  #This gives all 'Yes' and 'No'
  print('target varialbels', target_variables)
  variables = df[attribute].unique()    #This gives different features in that attribute (like 'Hot','Cold' in Temperature)
  print('variables',variables)
  entropy2 = 0
  for variable in variables:
      entropy = 0
      for target_variable in target_variables:
          num = len(df[attribute][df[attribute]==variable][df[Class] ==target_variable])
          den = len(df[attribute][df[attribute]==variable])
          fraction = num/(den+eps)
          entropy += -fraction*log(fraction+eps)
      fraction2 = den/len(df)
      entropy2 += -fraction2*entropy
  return abs(entropy2)


def find_winner(df):
    Entropy_att = []
    IG = []
    for key in df.keys()[:-1]:
#         Entropy_att.append(find_entropy_attribute(df,key))
        IG.append(find_entropy(df)-find_entropy_attribute(df,key))
        print('entropy result for ', key, find_entropy(df)-find_entropy_attribute(df,key) )
    print('returned value of find winner', df.keys()[:-1][np.argmax(IG)])
    return df.keys()[:-1][np.argmax(IG)]
  
  
def get_subtable(df, node,value):
  return df[df[node] == value].reset_index(drop=True)


def buildTree(df,tree=None): 
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name
    print('Class', Class)
    #Here we build our decision tree

    #Get attribute with maximum information gain
    node = find_winner(df)
    print('node', node)
    
    #Get distinct value of that attribute e.g Salary is node and Low,Med and High are values
    attValue = np.unique(df[node])
    
    #Create an empty dictionary to create tree    
    if tree is None:                    
        tree={}
        tree[node] = {}
    
   #We make loop to construct a tree by calling this function recursively. 
    #In this we check if the subset is pure and stops if it is pure. 
    print('attValue',attValue)
    for value in attValue:
        print('value', value)
        subtable = get_subtable(df,node,value)
        print('subtable', subtable)
        # clValue,counts = np.unique(subtable['Eat'],return_counts=True) 
        clValue,counts = np.unique(subtable['PlayTennis'],return_counts=True) 
        print('clValue, counts', clValue, counts)
        
        if len(counts)==1:#Checking purity of subset
            tree[node][value] = clValue[0]                                                    
        else:        
            tree[node][value] = buildTree(subtable) #Calling the function recursively 
                   
    return tree

In [22]:
buildTree(df, None)

Class PlayTennis
value in find entropy function No
value in find entropy function Yes
attribute Outlook
target varialbels ['No' 'Yes']
variables ['Sunny' 'Overcast' 'Rain']
value in find entropy function No
value in find entropy function Yes
attribute Outlook
target varialbels ['No' 'Yes']
variables ['Sunny' 'Overcast' 'Rain']
entropy result for  Outlook 0.24674981977443977
value in find entropy function No
value in find entropy function Yes
attribute Temperature
target varialbels ['No' 'Yes']
variables ['Hot' 'Mild' 'Cool']
value in find entropy function No
value in find entropy function Yes
attribute Temperature
target varialbels ['No' 'Yes']
variables ['Hot' 'Mild' 'Cool']
entropy result for  Temperature 0.029222565658955535
value in find entropy function No
value in find entropy function Yes
attribute Humidity
target varialbels ['No' 'Yes']
variables ['High' 'Normal']
value in find entropy function No
value in find entropy function Yes
attribute Humidity
target varialbels ['No' 'Ye

{'Outlook': {'Overcast': 'Yes',
  'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}},
  'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}}}